In [14]:
import sqlite3
import pickle
from more_itertools import unique_everseen
import cv2

In [15]:
conn = sqlite3.connect('data.dat')
cursor = conn.cursor()

cursor.execute('''SELECT ID, DATE FROM DATA''')
result = cursor.fetchall()
dates = [r[1] for r in result]
dates_unique = list(unique_everseen(dates))

In [16]:
dates_unique

['2016-04-16 10:51:14.288104',
 '2016-04-16 16:02:21.047555',
 '2016-04-17 20:30:53.262408',
 '2016-04-17 20:37:21.876155']

In [22]:
def modify(date):
    cursor.execute('SELECT ID, IMAGE FROM TRAIN WHERE DATE = ?', (date,))
    result = cursor.fetchall()
    images = [pickle.loads(i[1]) for i in result]
    ids = [i[0] for i in result]
    smaller = [cv2.resize(img, (0, 0), fx=0.2, fy=0.2, interpolation=cv2.INTER_LINEAR) \
              for img in images]
    new_img_str = [pickle.dumps(i) for i in smaller]
    for id_, img in zip(ids, new_img_str):
        cursor.execute('UPDATE TRAIN SET IMAGE = ? WHERE ID = ?', (img, id_))
    conn.commit()

In [20]:
cursor.execute('SELECT ID, IMAGE FROM TRAIN WHERE DATE = ?', (dates_unique[0],))
result = cursor.fetchall()

In [24]:
modify(dates_unique[1])